In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.svm import SVC

%matplotlib inline

In [2]:
## gt数据
path_gt = ["/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20200930_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201007_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201008_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201009_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201010_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201011_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201013_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201014_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201015_gt_data.csv",
         "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201017_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201023_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201024_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201025_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201027_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201028_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201030_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201031_gt_data.csv",
        "/mnt/data-2/evs_scenario_data/svm_data/2021_01_19_17_47_12/data/20201101_gt_data.csv"
       ]

In [3]:

# read cs
df = []
for i in range(len(path_gt)):
    df.append(pd.DataFrame(pd.read_csv(path_gt[i])))
len(df)

18

In [4]:
# sample all of the instances that label == 1
df_sample_true = []
for i in range(len(df)):
    df_sample_true.append(df[i].loc[df[i].label==1])

In [5]:
df_true = pd.concat(df_sample_true,axis=0,ignore_index=True)

In [6]:
df_true.shape[0]

238

In [7]:
# sample 800 instances which label is -1
df_sample_false = df[7].loc[df[7].label==-1]
df_sample_false = pd.concat([df_sample_false,df[2].loc[df[2].label==-1]],axis=0,ignore_index=True)
df_sample_false = shuffle(df_sample_false)[:800]
df_sample_false.shape

(800, 1221)

In [8]:
df_sample = pd.concat([df_true,df_sample_false],axis=0,ignore_index=True)
df_sample = shuffle(df_sample)

In [9]:
for i in range(10):
    name = ['stamp_'+str(i),'id_'+str(i),'uid_'+str(i),'target_front_obs_trackid_'+str(i),
            'target_front_obs_uid_'+str(i),'target_rear_obs_trackid_'+str(i),
            'target_rear_obs_uid_'+str(i),'front_obs_trackid_'+str(i),'front_obs_uid_'+str(i),
            'rear_obs_trackid_'+str(i),'rear_obs_uid_'+str(i)]
    for j in range(len(name)):
        df_sample.drop(name[j],axis=1, inplace=True)

In [10]:
def sizes_series(s):
    s['x_diff'] = s['x_9'] - s['x_0']
    s['y_diff'] = s['y_9'] - s['y_0']
    s['vx_diff'] = s['vx_9'] - s['vx_0']
    s['vy_diff'] = s['vy_9'] - s['vy_0']
    s['bx1_diff'] = s['bx1_9'] - s['bx1_0']
    s['bx2_diff'] = s['bx2_9'] - s['bx2_0']
    s['bx3_diff'] = s['bx3_9'] - s['bx3_0']
    s['bx4_diff'] = s['bx4_9'] - s['bx4_0']
    s['by1_diff'] = s['by1_9'] - s['by1_0']
    s['by2_diff'] = s['by2_9'] - s['by2_0']
    s['by3_diff'] = s['by3_9'] - s['by3_0']
    s['by4_diff'] = s['by4_9'] - s['by4_0']
    s['lat_diff'] = s['lat_dist_obs_to_tarlaneline_9'] - s['lat_dist_obs_to_tarlaneline_0']
    s['lo_otf_diff'] = s['lon_dist_obs_to_tarfrontobs_9'] - s['lon_dist_obs_to_tarfrontobs_0']
    s['lo_otr_diff'] = s['lon_dist_obs_to_tarrearobs_9'] - s['lon_dist_obs_to_tarrearobs_0']
    s['lo_of_diff'] = s['lon_dist_obs_to_frontobs_9'] - s['lon_dist_obs_to_frontobs_0']
    s['lo_or_diff'] = s['lon_dist_obs_to_rearobs_9'] - s['lon_dist_obs_to_rearobs_0']
    
    return s

In [11]:
df_sample = df_sample.apply(sizes_series, axis=1)

In [12]:
col = ['x_diff','y_diff','vx_diff','vy_diff','bx1_diff','bx2_diff','bx3_diff','bx4_diff','by1_diff','by2_diff',
      'by3_diff','by4_diff','lat_diff','lo_otf_diff','lo_otr_diff','lo_of_diff','lo_or_diff','label']
sample = df_sample.loc[:,col]

In [13]:
sample.head(10)

,x_diff,y_diff,vx_diff,vy_diff,bx1_diff,bx2_diff,bx3_diff,bx4_diff,by1_diff,by2_diff,by3_diff,by4_diff,lat_diff,lo_otf_diff,lo_otr_diff,lo_of_diff,lo_or_diff,label
1017,-3.393626,-0.040935,0.161188,-0.021217,-3.394193,-3.393085,-3.393059,-3.394167,-0.042593,-0.042601,-0.039277,-0.039269,1.803523e-02,1.241037,-3.393613,0.145950,NaN,-1.0
857,-0.156939,-0.039645,0.026907,-0.043235,-0.158779,-0.155143,-0.155099,-0.158734,-0.048729,-0.048738,-0.030561,-0.030552,-1.975180e-02,-0.891735,-0.156916,NaN,NaN,-1.0
470,-0.337133,-0.019869,0.092777,-0.022065,-0.337527,-0.336743,-0.336740,-0.337524,-0.021437,-0.021438,-0.018301,-0.018300,-3.660571e-03,0.445747,-0.337132,NaN,NaN,-1.0
885,-0.776320,0.004448,-1.328639,-0.013043,-0.776849,-0.775788,-0.775792,-0.776853,0.001795,0.001796,0.007101,0.007100,-9.371516e-03,NaN,-0.776323,NaN,NaN,-1.0
191,1.828348,-0.221708,0.050589,-0.110262,1.825635,1.830666,1.831062,1.826030,-0.229190,-0.229322,-0.214227,-0.214095,-2.044394e-01,NaN,1.828546,0.317971,NaN,1.0
519,-0.342464,-0.027332,0.104240,-0.034042,-0.343073,-0.341860,-0.341856,-0.343069,-0.029758,-0.029759,-0.024906,-0.024905,-4.860576e-03,0.445250,-0.342462,NaN,NaN,-1.0
886,0.331694,0.021713,0.064551,0.024603,0.332104,0.331272,0.331283,0.332115,0.023378,0.023375,0.020048,0.020050,1.960445e-02,NaN,0.331699,1.529364,NaN,-1.0
729,-1.735312,-0.029351,-0.311735,0.014714,-1.734705,-1.735900,-1.735919,-1.734725,-0.026666,-0.026662,-0.032036,-0.032040,1.319157e-02,2.766380,-1.735322,NaN,NaN,-1.0
247,-0.077934,0.021442,0.040770,-0.013332,-0.078283,-0.077581,-0.077585,-0.078288,0.020387,0.020389,0.022497,0.022495,-3.119656e-07,-0.299250,-0.077937,0.158539,NaN,-1.0
524,1.337876,0.029563,0.158551,-0.086530,1.336393,1.339436,1.339360,1.336317,0.023467,0.023486,0.035659,0.035640,2.713889e-02,-1.373581,1.337838,NaN,NaN,-1.0


In [14]:
sample = sample.fillna(500)

In [15]:
sample.head(10)

,x_diff,y_diff,vx_diff,vy_diff,bx1_diff,bx2_diff,bx3_diff,bx4_diff,by1_diff,by2_diff,by3_diff,by4_diff,lat_diff,lo_otf_diff,lo_otr_diff,lo_of_diff,lo_or_diff,label
1017,-3.393626,-0.040935,0.161188,-0.021217,-3.394193,-3.393085,-3.393059,-3.394167,-0.042593,-0.042601,-0.039277,-0.039269,1.803523e-02,1.241037,-3.393613,0.145950,500.0,-1.0
857,-0.156939,-0.039645,0.026907,-0.043235,-0.158779,-0.155143,-0.155099,-0.158734,-0.048729,-0.048738,-0.030561,-0.030552,-1.975180e-02,-0.891735,-0.156916,500.000000,500.0,-1.0
470,-0.337133,-0.019869,0.092777,-0.022065,-0.337527,-0.336743,-0.336740,-0.337524,-0.021437,-0.021438,-0.018301,-0.018300,-3.660571e-03,0.445747,-0.337132,500.000000,500.0,-1.0
885,-0.776320,0.004448,-1.328639,-0.013043,-0.776849,-0.775788,-0.775792,-0.776853,0.001795,0.001796,0.007101,0.007100,-9.371516e-03,500.000000,-0.776323,500.000000,500.0,-1.0
191,1.828348,-0.221708,0.050589,-0.110262,1.825635,1.830666,1.831062,1.826030,-0.229190,-0.229322,-0.214227,-0.214095,-2.044394e-01,500.000000,1.828546,0.317971,500.0,1.0
519,-0.342464,-0.027332,0.104240,-0.034042,-0.343073,-0.341860,-0.341856,-0.343069,-0.029758,-0.029759,-0.024906,-0.024905,-4.860576e-03,0.445250,-0.342462,500.000000,500.0,-1.0
886,0.331694,0.021713,0.064551,0.024603,0.332104,0.331272,0.331283,0.332115,0.023378,0.023375,0.020048,0.020050,1.960445e-02,500.000000,0.331699,1.529364,500.0,-1.0
729,-1.735312,-0.029351,-0.311735,0.014714,-1.734705,-1.735900,-1.735919,-1.734725,-0.026666,-0.026662,-0.032036,-0.032040,1.319157e-02,2.766380,-1.735322,500.000000,500.0,-1.0
247,-0.077934,0.021442,0.040770,-0.013332,-0.078283,-0.077581,-0.077585,-0.078288,0.020387,0.020389,0.022497,0.022495,-3.119656e-07,-0.299250,-0.077937,0.158539,500.0,-1.0
524,1.337876,0.029563,0.158551,-0.086530,1.336393,1.339436,1.339360,1.336317,0.023467,0.023486,0.035659,0.035640,2.713889e-02,-1.373581,1.337838,500.000000,500.0,-1.0


In [16]:
# 划分数据
X = sample.drop(['label'],axis=1,inplace=False)
Y = sample.label
feature_name = X.columns.tolist()

In [17]:
# 计算相关系数，选出与label相关性最大的前110个feature
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-110:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [18]:
cor_supp, cor_feature = cor_selector(X,Y)

/home/ziyi.liu/miniconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/ziyi.liu/miniconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [19]:
cor_feature

['lo_or_diff',
 'vx_diff',
 'lo_otf_diff',
 'vy_diff',
 'by4_diff',
 'by3_diff',
 'y_diff',
 'by1_diff',
 'by2_diff',
 'lo_of_diff',
 'lo_otr_diff',
 'bx1_diff',
 'bx4_diff',
 'x_diff',
 'bx2_diff',
 'bx3_diff',
 'lat_diff']

In [20]:
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=6)
chi_selector.fit(X_norm, Y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(chi_feature)

['x_diff', 'bx2_diff', 'bx3_diff', 'bx4_diff', 'lat_diff', 'lo_of_diff']


In [21]:
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=6, step=10, verbose=5)
rfe_selector.fit(X_norm, Y)
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
rfe_feature

Fitting estimator with 17 features.
Fitting estimator with 7 features.


['x_diff', 'bx1_diff', 'bx2_diff', 'bx3_diff', 'bx4_diff', 'lat_diff']

In [22]:
embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), '1.25*median')
embeded_lr_selector.fit(X_norm, Y)
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()

/home/ziyi.liu/miniconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass threshold=1.25*median as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [23]:
embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=6), threshold='1.25*median')
embeded_rf_selector.fit(X,Y)
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
embeded_rf_feature

['x_diff',
 'y_diff',
 'by1_diff',
 'by2_diff',
 'by3_diff',
 'by4_diff',
 'lat_diff',
 'lo_of_diff']

In [24]:
lgbc=LGBMClassifier(n_estimators=50, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.2*median',max_features=16)
embeded_lgb_selector.fit(X, Y)
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
embeded_lgb_feature

['x_diff',
 'y_diff',
 'bx1_diff',
 'bx2_diff',
 'bx4_diff',
 'by4_diff',
 'lat_diff',
 'lo_otr_diff']

self.clf = svm.SVC(C=self.C,
                                   kernel="rbf",
                                   gamma=self.gamma,
                                   class_weight="balanced",
                                   cache_size=1000,
                                   probability=True)

In [25]:
svm = SVC(C=1.0, kernel="linear", gamma=1.0, class_weight="balanced", cache_size=1000, probability=False)
svm_selector = SelectFromModel(svm, threshold='1.25*median')
svm_selector.fit(X, Y)
# print(svm_selector.estimator_.coef_)
svm_supp = svm_selector.get_support()
svm_fea = X.loc[:,svm_supp].columns.tolist()
svm_fea

['y_diff', 'vy_diff', 'bx4_diff', 'by1_diff', 'by2_diff', 'lat_diff']

In [26]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_supp, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support,'SVM':svm_supp})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(150)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,SVM,Total
1,lat_diff,True,True,True,True,True,True,True,7
2,x_diff,True,True,True,True,True,True,False,6
3,bx4_diff,True,True,True,True,False,True,True,6
4,bx2_diff,True,True,True,True,False,True,False,5
5,y_diff,True,False,False,False,True,True,True,4
6,lo_of_diff,True,True,False,True,True,False,False,4
7,bx3_diff,True,True,True,True,False,False,False,4
8,bx1_diff,True,False,True,True,False,True,False,4
9,by4_diff,True,False,False,False,True,True,False,3
10,by2_diff,True,False,False,False,True,False,True,3
